In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py
from collections import Counter
import numpy as np
from numpy import loadtxt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score

In [2]:
# f = h5py.File('mouse1sample1.hdf5','r')
# p_scores = pd.read_csv('merfish_M1S1_filtered_periph_scores.csv')
# p_scores = p_scores.loc[p_scores['annotation'] != 'unannotated']

In [34]:
# # time intensive step: get list of annotated cell_ids and annotation types 
# annotated_cells = []
# annotations = []
# for cell_id in f['cells']:
#     cell = f['cells'][cell_id]
#     ann = dict(cell.attrs)['annotation']
#     if ann != 'unannotated' and cell_id in list(p_scores['cell_id']):
#         annotated_cells.append(cell_id)
#         annotations.append(ann)
        
# annotations_count = Counter(annotations)
# print("Number of Annotated Samples:", len(annotated_cells))
# print("Num of Cell Types:", len(annotations_count))
# print("Max Number of Samples per Cell Type:", max(annotations_count.values()))
# print("Min Number of Samples per Cell Type:", min(annotations_count.values()))
# print("Average Number of Samples per Cell Type:", sum(annotations_count.values())/93)

In [35]:
# periphery data: shape (len(annotated_cells), num_genes)
# num_genes = (p_scores['gene']).unique().shape[0]
# cell_indices = {k: v for v, k in enumerate(list(p_scores['cell_id'].unique()))}
# gene_indices = {k: v for v, k in enumerate(list(p_scores['gene'].unique()))}

# periphery_data = np.zeros((len(annotated_cells), num_genes))
# for cell_id in annotated_cells:
#     subset = p_scores.loc[p_scores['cell_id'] == cell_id]
#     for index, s in subset.iterrows():
#         cell_index = cell_indices[cell_id]
#         gene_index = gene_indices[s['gene']]
#         periphery_data[cell_index][gene_index] = s['periphery_score']

# remove outlier genes
# expr = np.sum(np.square(periphery_data), axis=0)
# periphery_data = periphery_data[:,np.logical_and(expr >= np.percentile(expr, 1), expr <= np.percentile(expr, 99))]
# cv = np.std(periphery_data, axis=0) / np.mean(periphery_data, axis=0)
# periphery_data = periphery_data[:,np.logical_and(cv >= np.percentile(cv, 1), cv <= np.percentile(cv, 99))]
# np.save('periphery_data', periphery_data)

periphery_data = np.load('periphery_data.npy')

# remove outlier genes
expr = np.sum(np.square(periphery_data), axis=0)
periphery_data = periphery_data[:,np.logical_and(expr >= np.percentile(expr, 1), expr <= np.percentile(expr, 99))]
cv = np.std(periphery_data, axis=0) / np.mean(periphery_data, axis=0)
periphery_data = periphery_data[:,np.logical_and(cv >= np.percentile(cv, 1), cv <= np.percentile(cv, 99))]


In [36]:
# #image data: shape (len(annotated_cells), 224, 224))
# image_data = np.zeros(shape=(len(annotated_cells), 224, 224)).astype('uint8')
# fig = plt.figure(figsize=(4, 4), dpi=56)
# plt.gray()
# idx = 0
# for cell_id in annotated_cells:
#     cell = f['cells'][cell_id]
#     keys = list(cell['boundaries'].keys())
#     midpoint = keys[int(len(keys)/2)]
#     boundary = cell['boundaries'][midpoint]
#     xs = boundary[:,0]
#     ys = boundary[:,1]
#     plt.plot(xs,ys)
#     plt.axis('equal')
#     plt.xticks([])
#     plt.yticks([])
#     fig.canvas.draw()
#     fig_array = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
#     fig_array = fig_array.reshape(fig.canvas.get_width_height()[::-1] + (3,))
#     image_data[idx] = fig_array[:,:,0].astype('uint8')
#     idx += 1
#     plt.clf()
#     print(f"\r{idx}", end="")
  
# print(image_data.shape)
# np.save('image_data', image_data)

image_data = np.load('image_data.npy')
image_data = np.repeat(image_data[..., np.newaxis], 3, -1)

In [37]:
# one hot encoded label vectors
# annotation_indices = {k: v for v, k in enumerate(list(annotations_count.keys()))}
# labels = np.zeros((len(annotated_cells), len(annotation_indices)))
# for index in range(0, len(annotated_cells)):
#     cell = f['cells'][annotated_cells[index]]
#     ann = dict(cell.attrs)['annotation']
#     arr_index = annotation_indices[ann]
#     labels[index][arr_index] = 1
# print(labels.shape)
# np.save('labels', labels)

labels = np.load('labels.npy')
print(labels.shape)

(17312, 80)


In [38]:
indices = [x for x in range(labels.shape[0])]
train_indices, test = train_test_split(indices, test_size=0.3)
test_indices, val_indices = train_test_split(test, test_size=0.5)

image_train, periphery_train, labels_train = image_data[train_indices], periphery_data[train_indices], labels[train_indices]
image_test, periphery_test, labels_test = image_data[test_indices], periphery_data[test_indices], labels[test_indices]
image_val, periphery_val, labels_val = image_data[val_indices], periphery_data[val_indices], labels[val_indices]

print(image_train.shape, periphery_train.shape, labels_train.shape)
print(image_test.shape, periphery_test.shape, labels_test.shape)
print(image_val.shape, periphery_val.shape, labels_val.shape)

(12118, 224, 224, 3) (12118, 184) (12118, 80)
(2597, 224, 224, 3) (2597, 184) (2597, 80)
(2597, 224, 224, 3) (2597, 184) (2597, 80)


In [39]:
# initializing individual models + late fusion model
class WeightedAverage(tf.keras.layers.Layer):

    def __init__(self, n_output):
        super(WeightedAverage, self).__init__()
        self.W = tf.Variable(initial_value=tf.random.uniform(shape=[1,1,n_output], minval=0, maxval=1),
            trainable=True) # (1,1,n_inputs)

    def call(self, inputs):

        # inputs is a list of tensor of shape [(n_batch, n_feat), ..., (n_batch, n_feat)]
        # expand last dim of each input passed [(n_batch, n_feat, 1), ..., (n_batch, n_feat, 1)]
        inputs = [tf.expand_dims(i, -1) for i in inputs]
        inputs = tf.keras.layers.concatenate(axis=-1)(inputs) # (n_batch, n_feat, n_inputs)
        weights = tf.nn.softmax(self.W, axis=-1) # (1,1,n_inputs)
        # weights sum up to one on last dim
        return tf.reduce_sum(weights*inputs, axis=-1) # (n_batch, n_feat)
    
periphery_model = tf.keras.models.Sequential([tf.keras.layers.Input(periphery_data.shape[1]),
                                    tf.keras.layers.Dense(100, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(50, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(25, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(80, activation=tf.nn.softmax)])

image_model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(32,(3,3),padding='same', activation='relu',input_shape=(224, 224, 3)),
                                    tf.keras.layers.MaxPool2D((2,2)), 
                                    tf.keras.layers.Flatten(), 
                                    tf.keras.layers.Dropout(0.9),
                                    tf.keras.layers.Dense(64, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(80, activation=tf.nn.softmax)])

# model = tf.keras.models.Sequential()
# model.add(WeightedAverage(periphery_model.output, image_model.output))
# # weighted_average = tf.keras.layers.concatenate([tf.multiply(periphery_model.output, 0.1), tf.multiply(image_model.output, 0.1)], axis=0)
# # WA = WeightedAverage(n_output = 2)([periphery_model.output, image_model.output])
# # model = tf.keras.models.Model(inputs=[periphery_model.input, image_model.input], outputs = WA)
# model.summary()

In [40]:
# # compiling late fusion model with exponential decay lr
lr= tf.keras.optimizers.schedules.ExponentialDecay(
    0.0001,
    decay_steps=100000,
    decay_rate=0.95,
    staircase=True)

# early stopping to prevent overfitting 
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
periphery_model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=lr), metrics=['accuracy'])
# image_model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

In [41]:
# late fusion model training
periphery_model.fit(x=periphery_train, y=labels_train, epochs=300, validation_data=(periphery_val, labels_val))

Epoch 1/300
379/379 [==============================] - 2s 4ms/step - loss: 4.3058 - accuracy: 0.0353 - val_loss: 4.1105 - val_accuracy: 0.0608
Epoch 2/300
379/379 [==============================] - 1s 3ms/step - loss: 3.7405 - accuracy: 0.1033 - val_loss: 3.5343 - val_accuracy: 0.1171
Epoch 3/300
379/379 [==============================] - 1s 3ms/step - loss: 3.4783 - accuracy: 0.1192 - val_loss: 3.4443 - val_accuracy: 0.1290
Epoch 4/300
379/379 [==============================] - 1s 3ms/step - loss: 3.4050 - accuracy: 0.1708 - val_loss: 3.3782 - val_accuracy: 0.1941
Epoch 5/300
379/379 [==============================] - 1s 3ms/step - loss: 3.3348 - accuracy: 0.2069 - val_loss: 3.3054 - val_accuracy: 0.2126
Epoch 6/300
379/379 [==============================] - 1s 3ms/step - loss: 3.2586 - accuracy: 0.2139 - val_loss: 3.2302 - val_accuracy: 0.2141
Epoch 7/300
379/379 [==============================] - 1s 3ms/step - loss: 3.1823 - accuracy: 0.2174 - val_loss: 3.1561 - val_accuracy: 0.2229

In [42]:
image_model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=lr), metrics=['accuracy'])

In [43]:
# late fusion model training
image_model.fit(x=image_train, y=labels_train, epochs=10, validation_data=(image_val, labels_val))

2021-12-05 23:20:51.738906: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1824098304 exceeds 10% of free system memory.
2021-12-05 23:20:52.741816: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1824098304 exceeds 10% of free system memory.


Epoch 1/10


2021-12-05 23:20:54.516042: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


379/379 [==============================] - 32s 58ms/step - loss: 8.0575 - accuracy: 0.0897 - val_loss: 4.3529 - val_accuracy: 0.0874
Epoch 2/10
379/379 [==============================] - 21s 56ms/step - loss: 4.3384 - accuracy: 0.0923 - val_loss: 4.3237 - val_accuracy: 0.1174
Epoch 3/10
379/379 [==============================] - 21s 56ms/step - loss: 4.3096 - accuracy: 0.1180 - val_loss: 4.2951 - val_accuracy: 0.1174
Epoch 4/10
379/379 [==============================] - 21s 56ms/step - loss: 4.2814 - accuracy: 0.1180 - val_loss: 4.2672 - val_accuracy: 0.1174
Epoch 5/10
379/379 [==============================] - 21s 56ms/step - loss: 4.2539 - accuracy: 0.1180 - val_loss: 4.2400 - val_accuracy: 0.1174
Epoch 6/10
379/379 [==============================] - 21s 56ms/step - loss: 4.2270 - accuracy: 0.1180 - val_loss: 4.2133 - val_accuracy: 0.1174
Epoch 7/10
379/379 [==============================] - 21s 56ms/step - loss: 4.2008 - accuracy: 0.1180 - val_loss: 4.1874 - val_accuracy: 0.1174
Epo

In [108]:
# evaluation 
image_prediction = image_model.predict(image_test)
periphery_prediction = periphery_model.predict(periphery_test)
late_fusion_prediction = 0.999*periphery_prediction + 0.001*image_prediction

In [109]:
y_pred=np.argmax(late_fusion_prediction, axis=1)
y_test=np.argmax(labels_test, axis=1)

In [110]:
from sklearn.metrics import balanced_accuracy_score, accuracy_score
balanced_accuracy_score(y_test, y_pred)

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1854: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


0.22458205467128478

In [111]:
accuracy_score(y_test, y_pred)

0.536003080477474